In [ ]:
from lively import PositionLivelinessObjective, Solver, Translation, Rotation, Transform, SmoothnessMacroObjective, BoxShape, CollisionSettingInfo, ScalarRange, OrientationLivelinessObjective
from lxml import etree



#constructor not found 
translation = Translation(x=1.0, y =2.0, z=3.0)
rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0)
transform = Transform(translation=translation, rotation=rotation)
box = BoxShape(name="Table",frame="world",physical=True,x=2,y=1,z=1.2,local_transform = Transform.identity())
collision = CollisionSettingInfo(d_max = 0.1, r = 0.0, a_max = 2.0, time_budget = 100, timed = False)

# Read the xml file into a string
xml_file = '../../tests/basic.xml'
tree = etree.parse(xml_file)
xml_string = etree.tostring(tree).decode()

#print(xml_string)
# Instantiate a new solver
solver = Solver(
  urdf=xml_string, # Full urdf as a string
  objectives={
      # An example objective (smoothness macro)
      "smoothness":SmoothnessMacroObjective(name="MySmoothnessObjective",weight=5,joints=True,origin=False,links=True),
      "collision": CollisionAvoidanceObjective(name="MyCollisionAvoidanceObjective", weight=5),
      "positionLiveliness": PositionLivelinessObjective(name="MyPosLivelinessObjective",link="l_shoulder1",frequency=7,weight=25),
      "orientationLiveliness": OrientationLivelinessObjective(name="MyOrtLivelinessObjective", link="l_shoulder1",frequency=7,weight=25)
      
  },
  root_bounds=[
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0), # Translational, (x, y, z)
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0)  # Rotational, (r, p, y)
  ],
  shapes=[
      box
  ],
  collision_settings = collision

)

# Run solve to get a solved state
state = solver.solve(goals= translation,weights = {},time = 0.0)
# Log the initial state
print(state)